In [2]:
import numpy as np
import torch
from PIL import Image #处理图像
from torch.nn import functional as F    #提供各种函数，如激活函数、池化操作等。
from tqdm import tqdm   #显示循环进度条
import onnx
import onnxruntime
from onnx import numpy_helper
import os
import time

In [3]:
################### float32数据转int8模块  ##############
def float32_to_int8(float32_array): 
   # 定义量化范围（最小和最大值）
    min_val = np.min(float32_array)
    max_val = np.max(float32_array)

    # 计算缩放因子和零点
    scale = (max_val - min_val) / 127.0  # 127 是有符号 int8 的最大值
    zero_point = np.round(0 - min_val / scale)

    # 使用缩放因子和零点进行量化
    int8_array = np.round(float32_array / scale + zero_point).astype(np.int8)

    return int8_array   

In [4]:
#测时间
def bitlet_time(arr_w, arr_a):# arr_w =8x8   arr_a =8x8
    
    arr_w=float32_to_int8(arr_w)

    arr_a=float32_to_int8(arr_a)
################### 添加时间测试模块  ##############

    mantis = np.empty((8,64))
    time = 0

    weight_int8 = [""] * 64
    active_int8 = [""] * 64
    a_sign = [""] * 64
    w_sign = [""] * 64
    sign   = [""] * 64
                            
    sign = [""] * 64
    for b in range(64): 
        weight_int8[b] = '{0:08b}'.format(abs(arr_w[b]))
        active_int8[b] ='{0:08b}'.format(abs(arr_a[b]))   
        
        if arr_w[b] >= 0:
                w_sign[b] = 0
        else:
                w_sign[b] = 1

        if arr_a[b] >= 0:
                a_sign[b] = 0
        else:
                a_sign[b] = 1
                
        if w_sign[b]^a_sign[b]:
                sign[b] = 1
        else:
                sign[b] = 0
                
    w_manti_list = []
    a_manti_list = []

    for c in range(64):
            w_manti_list.append(weight_int8[c])
            a_manti_list.append(active_int8[c])


    
    for i in range(64):
        for j in range(8):
            mantis[j][i] = w_manti_list[j]


    ones = np.empty((8,4))
    for i in range(8):
        for k in range(4):
            one = 0
            for j in range(8):
                if mantis[i][32+8*k+j]:
                    one = one + 1 
            ones[i][k] = one

    for i in range(8):
        for k in range(4):
            if (ones[i][k] == 0 or ones[i][k] == 1):
                ones[i][k] = k+6
            elif (ones[i][k] == 2):
                ones[i][k] = k+7
            elif (ones[i][k] == (3 or 4)):
                ones[i][k] = k+8
            else:
                ones[i][k] = k+9

    for i in range(8):
        for k in range(3):
            for j in range(3-k):
                if(ones[i][k] == ones[i][k+j+1]):
                    ones[i][k] = ones[i][k] + 1

    for i in range(8):
        for k in range(3):
            for j in range(3-k):
                if(ones[i][k] == ones[i][k+j+1]):
                    ones[i][k] = ones[i][k] + 1

    for i in range(8):
        for k in range(3):
            for j in range(3-k):
                if(ones[i][k] == ones[i][k+j+1]):
                    ones[i][k] = ones[i][k] + 1

    time = ones[0][0]
    for i in range(8):
        for k in range(4):
            if(time < ones[i][k]):
                time = ones[i][k]


    
    
    return time+5
################### 添加时间测试模块  ##############

In [5]:
#乘加操作
def Mac_64(arr_w, mac_a64):
    weight_size = int(arr_w.size/1000)
    mac_time = 0

    for i in range(0, weight_size, 64):
        mac_time = mac_time + bitlet_time(arr_w[i:i+64],mac_a64)
    return mac_time

In [6]:
#卷积
def Conv_33(arr_w, arr_a, w_shape, a_shape, s):

    if(arr_w.size % 64 != 0):
        arr_w = np.pad(arr_w,(0,(64 - arr_w.size % 64)))

    yolo_time = 0
    mac_a64 = np.empty(64)
    cnt = 0

    for i in range(0 , (a_shape[3] - 2), s): 
        print("row = ",i)           #row
        for j in range(0 , (a_shape[2] - 2), s):        #channel
            for k in range(a_shape[1]):        #col
                for c in range(3):
                    for r in range(3):
                        mac_a64[cnt] =  arr_a[i+c,j+r,k]
                        cnt = cnt + 1
                        if(cnt == 64):
                            cnt = 0
                            yolo_time = yolo_time + Mac_64(arr_w, mac_a64)
            print(j , end = "|")

    return yolo_time

In [7]:
#卷积
def Conv_11(arr_w, arr_a, w_shape, a_shape):

    if(arr_w.size % 64 != 0):
        arr_w = np.pad(arr_w,(0,(64 - arr_w.size % 64)))

    yolo_time = 0
    mac_a64 = np.empty(64)
    cnt = 0

    for i in range(a_shape[3]):
        print("row = ",i)            #row
        for j in range(a_shape[2]):        #channel
            for k in range(a_shape[1]):        #col
                mac_a64[cnt] =  arr_a[i,j,k]
                cnt = cnt + 1
                if(cnt == 64):
                    cnt = 0
                    yolo_time = yolo_time + Mac_64(arr_w, mac_a64)
            print(j , end = "| ")

    return yolo_time

In [8]:
import numpy as np
import torch
from PIL import Image #处理图像
from torch.nn import functional as F    #提供各种函数，如激活函数、池化操作等。
from tqdm import tqdm   #显示循环进度条
import onnx
import onnxruntime
from onnx import numpy_helper
import os
import time

In [9]:
################### 提取weight模块  ##############
def extract_onnx_weight(onnx_path):
    model = onnx.load(onnx_path)

    weights, names = [], []
    for t in model.graph.initializer:
        weights.append(numpy_helper.to_array(t))
        names.append(t.name)
        
    onnx_weight = dict()
    for name, weight in zip(names, weights):
        onnx_weight[name] = weight
    return onnx_weight

In [10]:
if __name__ == '__main__':

    IMG_PATH = 'dog.jpg'
    ONNX_PATH = 'deit_quant_0519.onnx'
    OUT_TXT_PATH = "data.txt"

    
    w = ["1219_quantized","1222_quantized", "1223_quantized", "1224_quantized",
         "1225_quantized","1228_quantized", "1229_quantized", "1230_quantized",
         "1231_quantized","1234_quantized", "1235_quantized", "1236_quantized", 
         "1237_quantized","1240_quantized", "1241_quantized", "1242_quantized", 
         "1243_quantized","1246_quantized", "1247_quantized", "1248_quantized", 
        "1249_quantized", "1252_quantized", "1253_quantized", "1254_quantized",
        "1255_quantized", "1258_quantized","1259_quantized", "1260_quantized",
        "1261_quantized", "1264_quantized", "1265_quantized", "1266_quantized",
        "1267_quantized","1270_quantized", "1271_quantized", "1272_quantized",
        "1273_quantized","1276_quantized", "1277_quantized", "1278_quantized", 
		"1279_quantized","1282_quantized", "1283_quantized", "1284_quantized",
        "1285_quantized","1288_quantized", "1289_quantized"]

    a = ["193_quantized", "237_quantized", "252_quantized", "263_quantized",
         "278_quantized", "322_quantized", "337_quantized", "348_quantized", 
         "363_quantized","407_quantized", "422_quantized", "433_quantized",
         "448_quantized","492_quantized", "507_quantized", "518_quantized",
         "533_quantized","577_quantized", "592_quantized", "603_quantized", 
		"618_quantized","662_quantized", "677_quantized", "688_quantized",
        "703_quantized","747_quantized", "762_quantized", "773_quantized",
        "788_quantized","832_quantized", "847_quantized", "858_quantized",
        "873_quantized","917_quantized", "932_quantized", "943_quantized",
        "958_quantized","1002_quantized", "1017_quantized", "1028_quantized", 
		"1043_quantized","1087_quantized", "1102_quantized", "1113_quantized",
        "1128_quantized","1172_quantized","1187_quantized" ]
    '''
    
    w = ["1219_quantized", "226_quantized", "225_quantized", "1222_quantized", "1223_quantized", "1224_quantized", "1225_quantized", "311_quantized", "310_quantized", "1228_quantized", 
        "1229_quantized", "1230_quantized", "1231_quantized", "396_quantized", "395_quantized", "1234_quantized", "1235_quantized", "1236_quantized", "1237_quantized", "481_quantized", 
        "480_quantized", "1240_quantized", "1241_quantized", "1242_quantized", "1243_quantized", "566_quantized", "565_quantized", "1246_quantized", "1247_quantized", "1248_quantized", 
        "1249_quantized", "651_quantized", "650_quantized", "1252_quantized", "1253_quantized", "1254_quantized", "1255_quantized", "736_quantized", "735_quantized", "1258_quantized", 
        "1259_quantized", "1260_quantized", "1261_quantized", "821_quantized", "820_quantized", "1264_quantized", "1265_quantized", "1266_quantized", "1267_quantized", "906_quantized", 
        "905_quantized", "1270_quantized", "1271_quantized", "1272_quantized", "1273_quantized", "991_quantized", "990_quantized", "1276_quantized", "1277_quantized", "1278_quantized", 
        "1279_quantized", "1076_quantized", "1075_quantized", "1282_quantized", "1283_quantized", "1284_quantized", "1285_quantized", "1161_quantized", "1160_quantized", "1288_quantized", 
        "1289_quantized" ]
    
    a = ["193_quantized", "221_quantized", "230_quantized", "237_quantized", "252_quantized", "263_quantized", "278_quantized", "306_quantized", "315_quantized", "322_quantized", 
        "337_quantized", "348_quantized", "363_quantized", "391_quantized", "400_quantized", "407_quantized", "422_quantized", "433_quantized", "448_quantized", "476_quantized", 
        "485_quantized", "492_quantized", "507_quantized", "518_quantized", "533_quantized", "561_quantized", "570_quantized", "577_quantized", "592_quantized", "603_quantized", 
        "618_quantized", "646_quantized", "655_quantized", "662_quantized", "677_quantized", "688_quantized", "703_quantized", "731_quantized", "740_quantized", "747_quantized", 
        "762_quantized", "773_quantized", "788_quantized", "816_quantized", "825_quantized", "832_quantized", "847_quantized", "858_quantized", "873_quantized", "901_quantized", 
        "910_quantized", "917_quantized", "932_quantized", "943_quantized", "958_quantized", "986_quantized", "995_quantized", "1002_quantized", "1017_quantized", "1028_quantized", 
        "1043_quantized", "1071_quantized", "1080_quantized", "1087_quantized", "1102_quantized", "1113_quantized", "1128_quantized", "1156_quantized", "1165_quantized", "1172_quantized", 
        "1187_quantized"]
    '''
    
    print(len(w))
    print(len(a))
    
    torch.manual_seed(999)
    #打印数组元素的精度设置为10位小数（precision=10）
    # 并且不限制数组的打印阈值（threshold=np.inf）
    torch.set_printoptions(precision=10,sci_mode=True)
    torch.set_printoptions(threshold=np.inf)

    # activation
    img = Image.open(IMG_PATH).convert("RGB")
    img = np.asarray(img, np.float32)/255.0
    img= img.transpose(2,0,1).reshape(1,3,224,224)
    

    # weight
    onnx_path_list = [ONNX_PATH]
    
    out_file = OUT_TXT_PATH
    if os.path.exists(out_file):
        os.remove(out_file)
        print("文件已存在并已删除。")
    else:
        print("文件不存在。")

    final_time = np.empty((100))
    mac_cnt = np.empty((100))

    for layerX in range(len(w)):
        start_time = time.time()
        print("This is layer "+ str(layerX))

        print("w[layerX] ",w[layerX])
        WEIGHT_NAME = str(w[layerX])        #输入w
        MED_LAYER_NAME = str(a[layerX])      #输入a
        print("WEIGHT_NAME = "+WEIGHT_NAME)
        print("MED_LAYER_NAME = "+MED_LAYER_NAME)
        deit_weight = extract_onnx_weight(ONNX_PATH)
        select_weight = WEIGHT_NAME # "1138"
        weight = torch.from_numpy(deit_weight[select_weight])
        

        
        # activate
        intermediate_layer_name = MED_LAYER_NAME  
        model = onnx.load(ONNX_PATH)
        for node in model.graph.node:
            for output in node.output:
                if(output==intermediate_layer_name):
                    model.graph.output.extend([onnx.ValueInfoProto(name=output)])
        session = onnxruntime.InferenceSession(model.SerializeToString())
        # model input
        inputs = {'x': img}
        # infer
        intermediate_layer_output = session.run([intermediate_layer_name], inputs)[0]

        act = torch.from_numpy(intermediate_layer_output)
        ######################################################################
        w_shape = weight.shape 
        a_shape = act.shape

        print('w_shape:',weight.shape)
        print('a_shape:',act.shape)

        if(weight.ndim == 2):
            weight = weight.reshape(1,w_shape[0],w_shape[1])
        else:
            weight = weight.reshape(w_shape[1],w_shape[2],w_shape[3])
      
        if(act.ndim == 4):
            act = act.reshape(a_shape[1],a_shape[2],a_shape[3])
            print(act.shape)
        
        print('after modify:')
        print('w_shape:',weight.shape)
        print('a_shape:',act.shape)
        w_shape = weight.shape 
        a_shape = act.shape



        a_64 = np.empty(64)
        w_64 = np.empty(64)
        cnt = 0
        layer_time = 0

        #print(a_shape[2])
        #print(w_shape[1])
        assert (a_shape[2] == w_shape[1]), "error !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!."
        m_cnt = 0
        
        #在嵌套的循环中，对权重和激活参数执行一些操作，
        # 包括从它们中提取数据，然后将它们传递给 bitlet_time 函数，以计算时间和MAC计数。
        for k1 in range(w_shape[0]):
            for k2 in range(a_shape[0]):
                for k3 in range(a_shape[1]):
                    #print(k3,end = " ")
                    for k4 in range(int(w_shape[2]/10)):
                        for k5 in range(a_shape[2]):
                            a_64[cnt] = act[k1][k3][k5]
                            w_64[cnt] = weight[k2][k5][k4]
                            cnt = cnt + 1
                            
                            if(cnt == 64):
                                cnt = 0
                                m_cnt = m_cnt + 1
                                #print("a_64 =",a_64)
                                #print("w_64 =",w_64)
                                layer_time = layer_time + bitlet_time(a_64,w_64)
                print("")
        #计算每一对权重和激活参数的时间和MAC计数，
        # 并将这些值存储在 final_time 和 mac_cnt 数组中。
        
        end_time = time.time()
        final_time[layerX] = layer_time
        mac_cnt[layerX] = m_cnt
        print(layer_time)
        print(m_cnt)
        print("cp_time",end_time - start_time)


47
47
文件不存在。
This is layer 0
w[layerX]  1219_quantized
WEIGHT_NAME = 1219_quantized
MED_LAYER_NAME = 193_quantized


/tmp/ipykernel_33284/246041954.py:93: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  weight = torch.from_numpy(deit_weight[select_weight])


w_shape: torch.Size([192, 576])
a_shape: torch.Size([1, 197, 192])
after modify:
w_shape: torch.Size([1, 192, 576])
a_shape: torch.Size([1, 197, 192])

572679.0
33687
cp_time 42.38214898109436
This is layer 1
w[layerX]  1222_quantized
WEIGHT_NAME = 1222_quantized
MED_LAYER_NAME = 237_quantized
w_shape: torch.Size([192, 192])
a_shape: torch.Size([1, 197, 192])
after modify:
w_shape: torch.Size([1, 192, 192])
a_shape: torch.Size([1, 197, 192])

190893.0
11229
cp_time 14.064704656600952
This is layer 2
w[layerX]  1223_quantized
WEIGHT_NAME = 1223_quantized
MED_LAYER_NAME = 252_quantized
w_shape: torch.Size([192, 768])
a_shape: torch.Size([1, 197, 192])
after modify:
w_shape: torch.Size([1, 192, 768])
a_shape: torch.Size([1, 197, 192])

763572.0
44916
cp_time 58.74886131286621
This is layer 3
w[layerX]  1224_quantized
WEIGHT_NAME = 1224_quantized
MED_LAYER_NAME = 263_quantized
w_shape: torch.Size([768, 192])
a_shape: torch.Size([1, 197, 768])
after modify:
w_shape: torch.Size([1, 768, 192]

In [11]:
import numpy as np

total_time = 0
total_mac_cnt = 0
for i in range(71):
    total_time = total_time + final_time[i]
    total_mac_cnt = total_mac_cnt + mac_cnt[i]
print("total_time is ", total_time,"total_mac_cnt(64) is ", total_mac_cnt)

total_time is  26724450.0 total_mac_cnt(64) is  1572060.0
